# Data Formatting

## Step One: Pull Millennial Count from Age Data

In [2]:
import pandas as pd
import geopandas as gpd

In [158]:
age_df = pd.read_csv('raw-data/age-data-by-neighborhood.csv', encoding='utf-16-le', sep='\t')
age_df.head()

,Dashboard Name,Page,ACS 5 Label,Table ID,Neighborhood Name,Description 1,Description 2,Description 3,Race/Ethnicity,Units,Universe,Reliability Level,Measure Names,Measure Values
0,Minneapolis Neighborhood Demographics,Age Range,2018-2022,B01001,Armatage,10 - 17 years,NaN,NaN,all,people,total population,Medium,Percent Universe MOE,0.0506
1,Minneapolis Neighborhood Demographics,Age Range,2018-2022,B01001,Armatage,10 - 17 years,NaN,NaN,all,people,total population,Medium,Percent Universe,0.1360
2,Minneapolis Neighborhood Demographics,Age Range,2018-2022,B01001,Armatage,10 - 17 years,NaN,NaN,all,people,total population,Medium,Margin of Error,295.0000
3,Minneapolis Neighborhood Demographics,Age Range,2018-2022,B01001,Armatage,10 - 17 years,NaN,NaN,all,people,total population,Medium,Estimate,734.0000
4,Minneapolis Neighborhood Demographics,Age Range,2018-2022,B01001,Armatage,10 - 17 years,NaN,NaN,all,people,total population,Medium,Coefficient of Variation,0.2440


In [16]:
age_estimates = age_df[age_df['Measure Names'] == 'Estimate'][['Neighborhood Name', 'Description 1', 'Measure Values']]
age_estimates.head()

,Neighborhood Name,Description 1,Measure Values
3,Armatage,10 - 17 years,734.0
8,Armatage,18 - 24 years,173.0
13,Armatage,25 - 34 years,835.0
18,Armatage,35 - 49 years,1562.0
23,Armatage,50 - 64 years,723.0


In 2025, the ages of millennials are between 29-44. This is in the middle of two categories, so we will have to do some math to seperate the estimated count of millennials. 

25-34: I need 29-34, subtracting 4 ages from the bracket, 4 ages out of 10 total, meaning I will use 60% of this category estimate.

35-49: I need 35-44, subtracting 5 ages from the bracket, 5 ages out of 15 total, meaning I will use 67% of this category estimate.

In [55]:
lower_age_bracket_estimate = (age_estimates
                              [age_estimates['Description 1'] == '25 - 34 years']
                              .groupby('Neighborhood Name')
                              ['Measure Values'].sum()
                                * .6)
lower_age_bracket_estimate.head()

Neighborhood Name
Armatage        501.0
Audubon Park    685.2
Bancroft        398.4
Beltrami        186.6
Bottineau       308.4
Name: Measure Values, dtype: float64

In [54]:
higher_age_bracket_estimate = (age_estimates
                              [age_estimates['Description 1'] == '35 - 49 years']
                              .groupby('Neighborhood Name')
                              ['Measure Values'].sum()
                                * .67)
higher_age_bracket_estimate.head()

Neighborhood Name
Armatage        1046.54
Audubon Park     960.11
Bancroft         429.47
Beltrami         146.06
Bottineau        228.47
Name: Measure Values, dtype: float64

In [92]:
total_counts = age_estimates.groupby('Neighborhood Name')['Measure Values'].sum()
total_counts.head()

Neighborhood Name
Armatage        5396.0
Audubon Park    5377.0
Bancroft        2710.0
Beltrami        1414.0
Bottineau       1520.0
Name: Measure Values, dtype: float64

In [148]:
millennial_counts = (lower_age_bracket_estimate + higher_age_bracket_estimate).round().rename_axis('BDNAME')
millennial_percent = pd.DataFrame(millennial_counts / total_counts).rename(columns={'Measure Values': 'MIL_PERCENT'})
millennial_percent.reset_index(inplace=True)
millennial_percent.head()

,BDNAME,MIL_PERCENT
0,Armatage,0.286879
1,Audubon Park,0.305933
2,Bancroft,0.305535
3,Beltrami,0.235502
4,Bottineau,0.353289


## Step 2: Connect to GeoJSON File

First, I need to make sure neighborhood names match before I join them.

In [159]:
neighborhood_gdf = gpd.read_file('raw-data/minneapolis-neighborhoods.geojson')
neighborhood_gdf.head()

,OBJECTID,INT_REFNO,PREFIX,UDI,SYMBOL_NAM,BDNAME,BDNUM,TEXT_NBR,geometry
0,1,-2144134800,REFNO,23152.00,WARDAREA,Phillips West,90,90,"POLYGON ((-93.263 44.961, -93.263 44.961, -93...."
1,2,-2144131300,REFNO,23187.00,WARDAREA,Downtown West,87,87,"POLYGON ((-93.26 44.983, -93.26 44.983, -93.26..."
2,3,-2144131200,REFNO,23188.00,WARDAREA,Downtown East,88,88,"POLYGON ((-93.245 44.979, -93.245 44.979, -93...."
3,4,0,REFNO,23189.00,WARDAREA,Ventura Village,89,89,"POLYGON ((-93.25 44.966, -93.25 44.966, -93.24..."
4,5,-2144137100,REFNO,23129.00,WARDAREA,Sumner - Glenwood,29,29,"POLYGON ((-93.288 44.989, -93.288 44.989, -93...."


In [144]:
gdf_names = set(neighborhood_gdf['BDNAME'])
df_names = set(millennial_percent['BDNAME'])

print(f'In neighboood file but no age data: {gdf_names - df_names}')
print(f'In data file but no neighborhood data: {df_names - gdf_names}')

In neighboood file but no age data: {'Humboldt Industrial Area'}
In data file but no neighborhood data: set()


Since 'Humboldt Industrial Area' doesn't have age data in that dataset, I will take an average of the four surrounding neighborhoods, which I found using the neighborhood dataset (https://opendata.minneapolismn.gov/datasets/cityoflakes::minneapolis-neighborhoods/about)

The four surrounding neighborhoods:
- Shingle Creek
- Lind - Bohanon
- Webber - Camden
- Victory

In [145]:
target_neighborhoods = ['Shingle Creek', 'Lind - Bohanon', 'Webber - Camden', 'Victory']

humboldt_neighbors = millennial_percent[millennial_percent['BDNAME'].isin(target_neighborhoods)]
humboldt_neighbors

,BDNAME,MIL_PERCENT
39,Lind - Bohanon,0.229563
67,Shingle Creek,0.214174
77,Victory,0.277113
79,Webber - Camden,0.233579


In [149]:
humboldt_row = {
    'BDNAME': 'Humboldt Industrial Area',
    'MIL_PERCENT': humboldt_neighbors['MIL_PERCENT'].mean()
}

millennial_percent = pd.concat([millennial_percent, pd.DataFrame([humboldt_row])], ignore_index=True, axis=0)
millennial_percent

,BDNAME,MIL_PERCENT
0,Armatage,0.286879
1,Audubon Park,0.305933
2,Bancroft,0.305535
3,Beltrami,0.235502
4,Bottineau,0.353289
...,...,...
82,Whittier,0.321809
83,Willard - Hay,0.227832
84,Windom,0.323186
85,Windom Park,0.304132


In [152]:
neighborhood_gdf = pd.merge(neighborhood_gdf, millennial_percent)
neighborhood_gdf

,OBJECTID,INT_REFNO,PREFIX,UDI,SYMBOL_NAM,BDNAME,BDNUM,TEXT_NBR,geometry,MIL_PERCENT
0,1,-2144134800,REFNO,23152.00,WARDAREA,Phillips West,90,90,"POLYGON ((-93.263 44.961, -93.263 44.961, -93....",0.225601
1,2,-2144131300,REFNO,23187.00,WARDAREA,Downtown West,87,87,"POLYGON ((-93.26 44.983, -93.26 44.983, -93.26...",0.370131
2,3,-2144131200,REFNO,23188.00,WARDAREA,Downtown East,88,88,"POLYGON ((-93.245 44.979, -93.245 44.979, -93....",0.256063
3,4,0,REFNO,23189.00,WARDAREA,Ventura Village,89,89,"POLYGON ((-93.25 44.966, -93.25 44.966, -93.24...",0.191825
4,5,-2144137100,REFNO,23129.00,WARDAREA,Sumner - Glenwood,29,29,"POLYGON ((-93.288 44.989, -93.288 44.989, -93....",0.248666
...,...,...,...,...,...,...,...,...,...,...
82,83,-2144138700,REFNO,23113.00,WARDAREA,Marshall Terrace,13,13,"POLYGON ((-93.278 45.028, -93.278 45.028, -93....",0.281103
83,84,-2144134900,REFNO,23151.00,WARDAREA,Whittier,51,51,"POLYGON ((-93.27 44.963, -93.27 44.963, -93.27...",0.321809
84,85,-2144134700,REFNO,23153.00,WARDAREA,Lyndale,53,53,"POLYGON ((-93.284 44.948, -93.283 44.948, -93....",0.307311
85,86,-2144135400,REFNO,23146.00,WARDAREA,East Bde Maka Ska,46,46,"POLYGON ((-93.302 44.948, -93.301 44.948, -93....",0.334332


In [160]:
neighborhood_gdf.to_file('formatted-age-data.geojson')